In [1]:
import xtrack as xt
import numpy as np
import scipy.constants as sc
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import mplcursors
# %matplotlib tk

import matplotlib
# prevent NoneType error for versions of matplotlib 3.1.0rc1+ by calling matplotlib.use()
# For more on why it's nececessary, see
# https://stackoverflow.com/questions/59656632/using-qt5agg-backend-with-matplotlib-3-1-2-get-backend-changes-behavior
matplotlib.use('qt5agg')

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from PyQt5.QtWidgets import QMainWindow, QApplication, QWidget, QTabWidget, QVBoxLayout
import matplotlib.pyplot as plt
import sys

class plotWindow():
    def __init__(self, parent=None):
        self.app = QApplication(sys.argv)
        self.MainWindow = QMainWindow()
        self.MainWindow.__init__()
        self.MainWindow.setWindowTitle("plot window")
        self.canvases = []
        self.figure_handles = []
        self.toolbar_handles = []
        self.tab_handles = []
        self.current_window = -1
        self.tabs = QTabWidget()
        self.MainWindow.setCentralWidget(self.tabs)
        self.MainWindow.resize(1280, 900)
        self.MainWindow.show()

    def addPlot(self, title, figure):
        new_tab = QWidget()
        layout = QVBoxLayout()
        new_tab.setLayout(layout)

        figure.subplots_adjust(left=0.05, right=0.99, bottom=0.05, top=0.91, wspace=0.2, hspace=0.2)
        new_canvas = FigureCanvas(figure)
        new_toolbar = NavigationToolbar(new_canvas, new_tab)

        layout.addWidget(new_canvas)
        layout.addWidget(new_toolbar)
        self.tabs.addTab(new_tab, title)

        self.toolbar_handles.append(new_toolbar)
        self.canvases.append(new_canvas)
        self.figure_handles.append(figure)
        self.tab_handles.append(new_tab)

    def show(self):
        self.app.exec_()

In [2]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta3.json')
line['qph_setvalue'] = 0.5
line['qpv_setvalue'] = 0.5
line_no_aper = xt.Line(elements=[], element_names=[])

for ee, nn in zip(line.elements, line.element_names):
    if xt.line._is_aperture(ee, line):
        line_no_aper.append_element(xt.Marker(), nn)
    else:
        line_no_aper.append_element(ee, nn)
        # We are missing the parents of the slices
        if hasattr(ee, 'parent_name'):
            # If the parent is not in the line, we add it
            if ee.parent_name not in line_no_aper.element_dict:
                line_no_aper.env.element_dict[ee.parent_name] = line.get(ee.parent_name)
line_no_aper.particle_ref = line.particle_ref


Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [3]:
# Calculate bucket height
tw_ref = line.twiss()
beta0 = line.particle_ref.beta0[0]
E = line.particle_ref.energy0[0]
q = line.particle_ref.q0
f_RF = 200e6
h = f_RF * line.get_length() / beta0 / sc.c
eta = tw_ref.slip_factor

V_RF = 3e6
bucket_height_3MV = np.sqrt(2*q*V_RF / (np.pi*h*abs(eta)*E*beta0**2))

V_RF = 4.5e6
bucket_height_4_5MV = np.sqrt(2*q*V_RF / (np.pi*h*abs(eta)*E*beta0**2))

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [4]:
# Perform a bunch of twisses for different delta0
tw = {}
beam_sizes = {}
nemitt = 2e-6
for delta in np.linspace(-0.0075, 0.0075, 101):
    this_delta = round(delta,6)
    tw[this_delta] = line_no_aper.twiss(delta0=delta)
    beam_sizes[this_delta] = tw[this_delta].get_beam_covariance(nemitt_x=nemitt, nemitt_y=nemitt)


Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [5]:
def get_aper_x(ee):
    if isinstance(ee, xt.LimitPolygon):
        raise NotImplementedError
    max_x = 100
    min_x = -100
    if hasattr(ee, 'max_x'):
        max_x = ee.max_x
    if hasattr(ee, 'min_x'):
        min_x = ee.min_x
    if hasattr(ee, 'a_squ'):
        max_x = min(np.sqrt(ee.a_squ), max_x)
        min_x = max(-np.sqrt(ee.a_squ), min_x)
    if hasattr(ee, '_shift_x'):
        max_x += ee._shift_x
        min_x += ee._shift_x
    return max_x, min_x

def get_aper_y(ee):
    if isinstance(ee, xt.LimitPolygon):
        raise NotImplementedError
    max_y = 100
    min_y = -100
    if hasattr(ee, 'max_y'):
        max_y = ee.max_y
    if hasattr(ee, 'min_y'):
        min_y = ee.min_y
    if hasattr(ee, 'b_squ'):
        max_y = min(np.sqrt(ee.b_squ), max_y)
        min_y = max(-np.sqrt(ee.b_squ), min_y)
    if hasattr(ee, '_shift_y'):
        max_y += ee._shift_y
        min_y += ee._shift_y
    return max_y, min_y

In [14]:
tt = line.get_table()
tt_aper = tt.rows[[ttt.startswith('Limit') for ttt in tt.element_type]].cols['name s']
tt_aper.aper_max_x, tt_aper.aper_min_x = np.array([list(get_aper_x(line[nn]))  for nn in tt_aper.name]).T
tt_aper.aper_max_y, tt_aper.aper_min_y = np.array([list(get_aper_y(line[nn]))  for nn in tt_aper.name]).T

In [8]:
n_limit = 20
delta = np.array(list(tw.keys()))
aper_L = np.array([(tt_aper.aper_max_x - tw[dd].rows[tt_aper.name].x) / beam_sizes[dd].rows[tt_aper.name].sigma_x for dd in delta]).T
aper_R = np.array([(tt_aper.aper_min_x - tw[dd].rows[tt_aper.name].x) / beam_sizes[dd].rows[tt_aper.name].sigma_x for dd in delta]).T
aper_L = aper_L[[np.any((aa < n_limit) & (aa > -n_limit)) for aa in aper_L]]
aper_R = aper_R[[np.any((aa < n_limit) & (aa > -n_limit)) for aa in aper_R]]

fig, ax = plt.subplots(figsize=(10, 6))

figs = []

# 1) Make a custom cyclic colormap: green → red → green
cmap = mcolors.LinearSegmentedColormap.from_list(
    "green_red_cycle",
    ["tab:green", "tab:red", "tab:green"]
)
# 2) Normalize s so 0→7000 maps to 0→1
norm = mcolors.Normalize(vmin=0, vmax=line.get_length())

# plot left apertures
for idx, data in enumerate(aper_L):
    # grab the Line2D object
    fig, = ax.plot(data, delta, color=cmap(norm(tt_aper.s[idx])), picker=5)
    # stash the name on the fig’s "label" field
    fig.set_label(f"{tt_aper.name[idx]}  ({tt_aper.s[idx]}m)")
    figs.append(fig)

# plot right apertures
for idx, data in enumerate(aper_R):
    fig, = ax.plot(data, delta, color=cmap(norm(tt_aper.s[idx])), picker=5)
    fig.set_label(f"{tt_aper.name[idx]}  ({tt_aper.s[idx]}m)")
    figs.append(fig)

ax.set_xlim((-n_limit,n_limit))
ax.hlines(-bucket_height_4_5MV, -n_limit, n_limit, color='gray', linestyle='--')
ax.hlines(bucket_height_4_5MV, -n_limit, n_limit, color='gray', linestyle='--')
ax.hlines(-2*bucket_height_4_5MV, -n_limit, n_limit, color='lightgray', linestyle='--')
ax.hlines(2*bucket_height_4_5MV, -n_limit, n_limit, color='lightgray', linestyle='--')

# now attach the hover‐tootip
cursor = mplcursors.cursor(figs)
@cursor.connect("add")
def _(sel):
    sel.annotation.set_text(sel.artist.get_label())

plt.show()

In [ ]:
# for i in range(6):
#     fig, ax = plt.subplots(figsize=(20, 5))
#     delta = list(tw.keys())
#     for dd in delta:
#         if abs(dd) > 0.00375:
#             ax.plot(tw[dd].s, tw[dd].x + np.sign(dd)*2*i*beam_sizes[dd].sigma_x, 'r-')
#         else:
#             ax.plot(tw[dd].s, tw[dd].x + np.sign(dd)*2*i*beam_sizes[dd].sigma_x, 'g-')
#     line1, = ax.plot(tt_aper.s, tt_aper.aper_max_x, 'k-')
#     line2, = ax.plot(tt_aper.s, tt_aper.aper_min_x, 'k-')
#     ax.set_xlabel('s [m]')
#     ax.set_ylabel('Aperture $x$ [m]')
#     ax.set_title(f'Betatron {2*i}' + r'$\sigma_x$   $\delta$ within 1 bucket (green) and 2 buckets (red)')
#     cursor = mplcursors.cursor([line1, line2])
#     @cursor.connect("add")
#     def on_add(sel):
#         if hasattr(sel, 'index'):
#             raw_idx = sel.index
#         else:
#             raw_idx = sel.target.index
#         idx = int(raw_idx)
#         sel.annotation.set_text(tt_aper.name[idx])
#     plt.show()

In [33]:
pw = plotWindow()
for i in range(6):
    fig, ax = plt.subplots(figsize=(20, 4))
    delta = list(tw.keys())
    for dd in delta:
        if abs(dd) > 0.00375:
            ax.plot(tw[dd].s, tw[dd].x + np.sign(dd)*2*i*beam_sizes[dd].sigma_x, 'r-')
        else:
            ax.plot(tw[dd].s, tw[dd].x + np.sign(dd)*2*i*beam_sizes[dd].sigma_x, 'g-')
    line1, = ax.plot(tt_aper.s, tt_aper.aper_max_x, 'k-')
    line2, = ax.plot(tt_aper.s, tt_aper.aper_min_x, 'k-')
    ax.set_xlabel('s [m]')
    ax.set_ylabel('Aperture $x$ [m]')
    ax.set_title(f'Betatron {2*i}' + r'$\sigma_x$   $\delta$ within 1 bucket (green) and 2 buckets (red)')
    cursor = mplcursors.cursor([line1, line2])
    @cursor.connect("add")
    def on_add(sel):
        if hasattr(sel, 'index'):
            raw_idx = sel.index
        else:
            raw_idx = sel.target.index
        idx = int(raw_idx)
        sel.annotation.set_text(tt_aper.name[idx])
    pw.addPlot(f"{2*i}" + r'$\sigma_x$', fig)
pw.show()

: 

In [12]:
tt_aper.rows['qf.*']

Table: 0 rows, 6 cols

In [15]:
tt.rows['qf.*']

Table: 108 rows, 11 cols
name                 s element_type isthick isreplica parent_name iscollective       s_start ...
qf.10010             0 Quadrupole      True     False None               False             0
qf.10210       63.9954 Quadrupole      True     False None               False       63.9954
qf.10410       127.991 Quadrupole      True     False None               False       127.991
qf.10610       191.986 Quadrupole      True     False None               False       191.986
qf.10810       255.982 Quadrupole      True     False None               False       255.982
qf.11010       319.977 Quadrupole      True     False None               False       319.977
qf.11210       383.972 Quadrupole      True     False None               False       383.972
qf.11410       447.968 Quadrupole      True     False None               False       447.968
qf.11610       511.963 Quadrupole      True     False None               False       511.963
qf.11810       575.959 Quadrupole      Tr

In [32]:
fig, ax = plt.subplots(figsize=(20, 5))
lines = []
this_tw = tw[-0.0036].rows['qf.*']
line, = ax.plot(this_tw.s, this_tw.x, 'b.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'b.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'b.'); lines.append(line)
this_tw = tw[-0.0072].rows['qf.*']
line, = ax.plot(this_tw.s, this_tw.x, 'b.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'b.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'b.'); lines.append(line)
this_tw = tw[-0.0036].rows['qd.*']
line, = ax.plot(this_tw.s, this_tw.x, 'm.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'm.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'm.'); lines.append(line)
this_tw = tw[-0.0072].rows['qd.*']
line, = ax.plot(this_tw.s, this_tw.x, 'm.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'm.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'm.'); lines.append(line)
this_tw = tw[-0.0036].rows['mba.*']
line, = ax.plot(this_tw.s, this_tw.x, 'g.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'g.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'g.'); lines.append(line)
this_tw = tw[-0.0072].rows['mba.*']
line, = ax.plot(this_tw.s, this_tw.x, 'g.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'g.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'g.'); lines.append(line)
this_tw = tw[-0.0036].rows['mbb.*']
line, = ax.plot(this_tw.s, this_tw.x, 'r.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'r.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'r.'); lines.append(line)
this_tw = tw[-0.0072].rows['mbb.*']
line, = ax.plot(this_tw.s, this_tw.x, 'r.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'r.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'r.'); lines.append(line)
ax.hlines(-7.6e-2, 0, 7000, color='b', linestyle='--') # QF
ax.hlines(-4e-2, 0, 7000, color='m', linestyle='--') # QD
ax.hlines(-7.6e-2-2.2e-3, 0, 7000, color='g', linestyle='--') # MBA
ax.hlines(-7.6e-2+4.4e-3, 0, 7000, color='g', linestyle='--') # MBA
ax.hlines(-6.45e-2-2.2e-3, 0, 7000, color='r', linestyle='--') # MBB
ax.hlines(-6.45e-2+4.4e-3, 0, 7000, color='r', linestyle='--') # MBB

cursor = mplcursors.cursor(lines)
@cursor.connect("add")
def on_add(sel):
    if hasattr(sel, 'index'):
        raw_idx = sel.index
    else:
        raw_idx = sel.target.index
    idx = int(raw_idx)
    sel.annotation.set_text(tw[0].rows['q[df].*'].name[idx])
plt.show()

In [ ]:
offset_end_dipole=4.4e-3
offset_middle_dipole=2.2e-3

In [ ]:
mba_names = np.unique([ttt.split('_')[0].split('.')[:2] for ttt in tt.name if ttt.startswith('mba')])
tt.rows['mba.*_aper2']

Table: 3960 rows, 11 cols
name                           s element_type isthick isreplica parent_name iscollective ...
mba.10030..0_aper2       3.44499 LimitRect      False     False None               False
mba.10030..1_aper2       4.07099 LimitRect      False     False None               False
mba.10030..2_aper2       4.69699 LimitRect      False     False None               False
mba.10030..3_aper2         5.323 LimitRect      False     False None               False
mba.10030..4_aper2         5.949 LimitRect      False     False None               False
mba.10030..5_aper2         6.575 LimitRect      False     False None               False
mba.10030..6_aper2         7.201 LimitRect      False     False None               False
mba.10030..7_aper2         7.827 LimitRect      False     False None               False
mba.10030..8_aper2       8.45301 LimitRect      False     False None               False
mba.10030..9_aper2       9.07901 LimitRect      False     False None            

array(['10030', '10050', '10170', '10190', '10230', '10250', '10370',
       '10390', '10430', '10450', '10570', '10590', '10630', '10650',
       '10770', '10790', '10830', '10850', '10970', '10990', '11030',
       '11050', '11170', '11190', '11230', '11250', '11370', '11390',
       '11570', '11590', '12030', '12050', '12230', '12250', '12370',
       '12390', '12430', '12450', '12570', '12590', '12630', '12650',
       '12770', '12790', '12830', '12850', '12970', '12990', '13030',
       '13050', '13170', '13190', '13230', '13250', '13370', '13390',
       '13430', '13450', '13570', '13590', '20030', '20050', '20170',
       '20190', '20230', '20250', '20370', '20390', '20430', '20450',
       '20570', '20590', '20630', '20650', '20770', '20790', '20830',
       '20850', '20970', '20990', '21030', '21050', '21170', '21190',
       '21230', '21250', '21370', '21390', '21570', '21590', '22030',
       '22050', '22230', '22250', '22370', '22390', '22430', '22450',
       '22570', '225